In [16]:
# 1. Instalação da biblioteca de estatística
!pip install factor_analyzer

# 2. Importação de bibliotecas e ferramentas de upload
import pandas as pd
import numpy as np
import os
from google.colab import files, drive
from factor_analyzer import FactorAnalyzer

# 3. Upload do arquivo de mapeamento
print("Por favor, selecione o arquivo: 'associado_indicadores.xlsx '")
uploaded = files.upload()

# Identifica o nome do arquivo carregado para evitar erros de digitação
nome_real_arquivo = list(uploaded.keys())[0]
mapping = pd.read_csv(nome_real_arquivo)

print(f"\nSucesso! Arquivo '{nome_real_arquivo}' carregado e lido.")

Por favor, selecione o arquivo: 'associado_indicadores.xlsx '


Saving COPSOQ_BR_Ordenado_Associado_Indicadores.xlsx - perguntas-dimensões.csv.xlsx - perguntas-dimensões.csv to COPSOQ_BR_Ordenado_Associado_Indicadores.xlsx - perguntas-dimensões.csv.xlsx - perguntas-dimensões (1).csv

Sucesso! Arquivo 'COPSOQ_BR_Ordenado_Associado_Indicadores.xlsx - perguntas-dimensões.csv.xlsx - perguntas-dimensões (1).csv' carregado e lido.


In [18]:
# CONFIGURAÇÕES DE REPRODUTIBILIDADE
SEED = 42
np.random.seed(SEED)
N_AMOSTRA = 500

# GERAÇÃO DA BASE SIMULADA (TOY DATASET)
# Simula a estrutura latente para os 3 indicadores conforme a proposta metodológica [cite: 69, 70]
latent_factors = np.random.multivariate_normal(
    mean=[0, 0, 0],
    cov=[[1.0, 0.2, 0.2], [0.2, 1.0, 0.2], [0.2, 0.2, 1.0]],
    size=N_AMOSTRA
)

df_simulada = pd.DataFrame()
for idx, row in mapping.iterrows():
    item_id = f"Item_{row['Nº']}"
    f_idx = 0 if 'ISP' in row['Indicador'] else (1 if 'IAT' in row['Indicador'] else 2)

    # Gera dados com carga fatorial baseada na teoria dos macro-fatores [cite: 103, 106]
    raw = 0.7 * latent_factors[:, f_idx] + np.random.normal(0, 0.5, N_AMOSTRA)

    # Discretização para escala Likert 1-5 conforme as diretrizes do instrumento [cite: 67]
    scaled = (raw - raw.min()) / (raw.max() - raw.min())
    df_simulada[item_id] = np.round(scaled * 4 + 1).astype(int)

# ANÁLISE ESTATÍSTICA
fa = FactorAnalyzer(n_factors=3, method='principal', rotation='varimax')
fa.fit(df_simulada)

def calcular_alfa(df, indicador_nome):
    itens = mapping[mapping['Indicador'].str.contains(indicador_nome)]['Nº']
    cols = [f"Item_{i}" for i in itens]
    sub_df = df[cols]
    k = sub_df.shape[1]
    item_vars = sub_df.var(ddof=1).sum()
    total_var = sub_df.sum(axis=1).var(ddof=1)
    return (k / (k - 1)) * (1 - (item_vars / total_var))

# Resultados Finais
print("\n--- RESULTADOS DA VALIDAÇÃO (N=500) ---")
print(f"Alfa de Cronbach - ISP: {calcular_alfa(df_simulada, 'ISP'):.3f}")
print(f"Alfa de Cronbach - IAT: {calcular_alfa(df_simulada, 'IAT'):.3f}")
print(f"Alfa de Cronbach - NSP: {calcular_alfa(df_simulada, 'NSP'):.3f}")

# DOWNLOAD
nome_arquivo = 'base_simulada_500_respostas.csv'
df_simulada.to_csv(nome_arquivo, index=False)

from google.colab import files
print(f"\nProcessamento concluído. Iniciando download de: {nome_arquivo}")
files.download(nome_arquivo)


--- RESULTADOS DA VALIDAÇÃO (N=500) ---
Alfa de Cronbach - ISP: 0.966
Alfa de Cronbach - IAT: 0.914
Alfa de Cronbach - NSP: 0.917

Processamento concluído. Iniciando download de: base_simulada_500_respostas.csv


/usr/local/lib/python3.12/dist-packages/sklearn/utils/deprecation.py:151: FutureWarning: 'force_all_finite' was renamed to 'ensure_all_finite' in 1.6 and will be removed in 1.8.
  warnings.warn(


<IPython.core.display.Javascript object>

<IPython.core.display.Javascript object>